In [1]:
import numpy as np
import matplotlib as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

ModuleNotFoundError: No module named 'torch'

In [ ]:
#On définie le path vers nos images

custom_image_path = r'photos_usecase3'

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1= nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1= nn.ReLU()
        self.pool1= nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2= nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2= nn.ReLU()
        self.pool2= nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1  = nn.Linear(32*56*56,64)
        self.relu3= nn.ReLU()
        self.fc2  = nn.Linear(64,2) #2 output photos reportages & photos studios

    def forward(self, x):
        x= self.pool1(self.relu1(self.conv1(x)))
        x= self.pool2(self.relu2(self.conv2(x)))
        x= x.view(-1,32*56*56)
        x= self.relu3(self.fc1(x))
        x= self.fc2(x)
        return x

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
#Chargement du dataset
dataset= ImageFolder(root=custom_image_path,transform = transform)

In [ ]:
#On sépare le dataset entre ceux qui seront pour l'entrainement d'IA et ceux pour la validation (80/20)
train_size = int(0.8*len(dataset))
val_size = len(dataset)-train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset,[train_size,val_size])

In [ ]:
#On crée le chargement des données
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=batch_size)

In [ ]:
#On initialise le modele
model = SimpleCNN()

#On calcule l'efficacité du modele

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#Training loop

epochs= 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs,labels)
            val_loss+= loss.item() * images.size(0)
            _, preds = torch.max(outputs,1)
            val_corrects+= torch.sum(preds==labels.data)

    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = val_corrects.double() / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss: .4f}, Val loss: {val_loss: .4f}, Val accuracy: {val_accuracy: .4f}")

In [ ]:
#Sauvegarde le model actuel
torch.save(model.state_dict(), 'custom_cnn_model.pth')

In [ ]:
import os
from PIL import Image

# Initialisation des variables
pourcentage_studio = 0
pourcentage_reportage = 0
total_images = 0

# Chemin vers le dossier test
test_folder = r"Photos_Test"

# Classes du modèle
class_labels = ['photo reportage', 'photo studio']

# Parcours tous les fichiers et sous-dossiers dans le dossier test
for root, dirs, files in os.walk(test_folder):
    for filename in files:
        # Vérifie si le fichier est une image (en fonction des extensions)
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
            total_images += 1
            image_path = os.path.join(root, filename)
            
            # Charge l'image
            try:
                image = Image.open(image_path)
                image_tensor = transform(image).unsqueeze(0).to(device)
                
                # Prédire avec le modèle
                model.eval()
                with torch.no_grad():
                    outputs = model(image_tensor)
                    _, predicted = torch.max(outputs, 1)
                
                prediction = class_labels[predicted.item()]
                print(f"The image '{image_path}' is classified as: {prediction}")
                
                # Comptabilise les prédictions correctes pour 'photo studio'
                if prediction == 'photo studio':
                    pourcentage_studio += 1
                else:
                    pourcentage_reportage+=1
            except Exception as e:
                print(f"Error processing file '{image_path}': {e}")

# Affiche les résultats
if total_images > 0:
    print(f"The model detects photo studio out of {total_images} images nby: {pourcentage_studio * 100 / total_images:.2f}%")
    print(f"The model detects photo reportage out of {total_images} images by: {pourcentage_reportage * 100 / total_images:.2f}%")
else:
    print("No valid images found in the folder!")


In [ ]:
"""model.eval()

with torch.no_grad():
    outputs = model(image_tensor)
    _, predicted = torch.max(outputs, 1)

class_labels = ['photo reportage','photo studio']
prediction = class_labels[predicted.item()]

print(f"The image is classified as : {prediction}")"""